In [2]:
import pandas as pd

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [9]:
dep=dataset[['Purchased']]

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split( indep, dep, test_size=0.30, random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(x_train)
X_test=sc.transform(x_test)

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
from sklearn.model_selection import GridSearchCV
param_grid = {'solver':['newton-cg','lbfgs','liblinear','saga'],
             'penalty':['l2']}
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [15]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [16]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("the f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)

the f1_macro value for best parameter{'penalty': 'l2', 'solver': 'newton-cg'}: 0.8906190214115367


In [17]:
print("the confusion matrix:\n",cm)

the confusion matrix:
 [[74  5]
 [ 8 33]]


In [18]:
print("the report:\n",clf_report)

the report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9481321395492436

In [20]:
table=pd.DataFrame.from_dict(re)

In [21]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015626,0.000001,0.018752,0.006250,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
1,0.015626,0.000002,0.015624,0.000002,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
2,0.003125,0.006251,0.021879,0.007655,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
3,0.009373,0.007653,0.015630,0.000001,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1


In [35]:
result=grid.predict([[19,19000,1]])
result

array([1], dtype=int64)